In [1]:
import torch 
import torch_geometric
import numpy as np
from fuzzywuzzy import fuzz

torch.seed()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device {device}")

/home/prerak/anaconda3/envs/kg/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/prerak/anaconda3/envs/kg/lib/python3.11/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Using device cpu


In [2]:
from generate_graph import graph_to_ttl
G = graph_to_ttl()


In [5]:
nodes = set()
relations = set()

for stmt in G:
    nodes.add(str(stmt[0]))
    nodes.add(str(stmt[2]))
    relations.add(str(stmt[1]))

nodes = list(nodes)
relations = list(relations)

In [6]:
print(nodes)
print(relations)

['actor:Billy_Bob_Thornton', 'actor:Calum_Worthy', 'actor:Michele_Valley', 'movie:The_Last_Face', 'movie:Midnight_in_Paris', 'actor:Tom_Mison', 'movie:Star_Trek_Into_Darkness', 'movie:The_House_Bunny', 'actor:Gwyneth_Paltrow', 'movie:Dark_Shadows', 'actor:Jorge_Garcia', 'actor:Kevin_Chamberlin', 'actor:A.C._Peterson', 'director:Gareth_Edwards', 'actor:Laurence_Fishburne', 'actor:Thomas_Lennon', 'actor:Carly_Rae_Jepsen', 'actor:Ben_Affleck', 'actor:Trey_Songz', 'actor:Briana_Evigan', 'director:Greg_Mottola', 'director:Evan_Goldberg', 'actor:Maribel_Verdú', 'movie:Tropic_Thunder', 'movie:Mission:_Impossible_III', 'actor:Stuart_Graham', 'actor:Mia_Wasikowska', 'actor:Bijou_Phillips', 'director:Tim_Burton', 'movie:Oblivion', 'actor:Scott_Lawrence', 'movie:The_Prestige', 'actor:Eddie_Murphy', 'actor:David_Harbour', 'movie:Entourage', 'director:Baltasar_Kormákur', 'actor:Shia_LaBeouf', "actor:John_D'Leo", 'actor:Naomi_Battrick', 'actor:Zach_Galifianakis', 'director:Michael_Dudok_de_Wit', 'ac

In [67]:
def fuzzsearch(w, is_node):
    w = w.strip()
    if(w.count("?") != 0):
        return w
    if is_node:
        scores = list(map(lambda x : fuzz.ratio(x, w), nodes))
        idxs = np.flip(np.argsort(scores))
        return nodes[idxs[0]]
    else:
        scores = list(map(lambda x : fuzz.ratio(x, w), relations))
        idxs = np.flip(np.argsort(scores))
        return relations[idxs[0]]

In [70]:
Q = '''?year1[("Elysium", "movie2year", ?year1), ("Elysium", "movie2actor", "Matt Damon"), ("Elysium", "movie2actor", "Jodie Foster"), ("Elysium", "movie2genre", "science fiction")]'''

import re

def get_sparSQL(query):
    # regex = r"?[^,^)]+"
    # variables = re.findall(regex, query)
    # print(variables)
    regex = r"\(([^)]+)\)"
    relations = re.findall(regex, query)
    
    sparSQL = "SELECT DISTINCT \nWHERE {\n"
    
    for i in relations:
        r = transform_relation(i)
        if(r != None):
            sparSQL += r
    sparSQL += "}"
    print(sparSQL)
    

def transform_relation(relation):
    if(relation.count('?') == 0):
        return None
    q = ""
    count = 0
    for term in relation.split(","):
        q += f"{fuzzsearch(term, count != 1)}"
        if(count != 2):
            q += " "
        count += 1
    q += ".\n"
    return q
    
get_sparSQL(Q)   

SELECT DISTINCT 
WHERE {
movie:Elysium relation:movie2year ?year1.
}


In [60]:
q = """
SELECT DISTINCT ?movie1 ?actor1 
WHERE {
?movie1 relation:movie2year year:2010.
?movie1 relation:movie2actor ?actor1.
movie:Inception relation:movie2actor ?actor1.
}
"""

qres = G.query(q)

In [61]:
for res in qres:
    print(str(res[0]))

movie:Inception
movie:Inception
movie:Inception
movie:Inception
movie:Shutter_Island
movie:Super
